In [59]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [60]:
base_dir = 'dataset'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

In [61]:
img_height, img_width = 48, 48
batch_size = 32

In [62]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [63]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [64]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 28709 images belonging to 7 classes.


In [66]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 7178 images belonging to 7 classes.


In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(32, (3, 3), input_shape=(img_height, img_width, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(train_generator.num_classes, activation='softmax'))

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)


In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size,
    epochs=50,
    callbacks=[early_stopping, model_checkpoint]
)

Epoch 1/50
897/897 ━━━━━━━━━━━━━━━━━━━━ 50s 53ms/step - accuracy: 0.2386 - loss: 1.8359 - val_accuracy: 0.2559 - val_loss: 1.7733
Epoch 2/50
897/897 ━━━━━━━━━━━━━━━━━━━━ 0s 41us/step - accuracy: 0.2188 - loss: 1.8317 - val_accuracy: 0.1000 - val_loss: 2.2316
Epoch 3/50
897/897 ━━━━━━━━━━━━━━━━━━━━ 47s 52ms/step - accuracy: 0.2562 - loss: 1.7887 - val_accuracy: 0.2881 - val_loss: 1.7175
Epoch 4/50
897/897 ━━━━━━━━━━━━━━━━━━━━ 0s 26us/step - accuracy: 0.1875 - loss: 1.8651 - val_accuracy: 0.3000 - val_loss: 1.7889
Epoch 5/50
897/897 ━━━━━━━━━━━━━━━━━━━━ 54s 60ms/step - accuracy: 0.2714 - loss: 1.7589 - val_accuracy: 0.3050 - val_loss: 1.6894
Epoch 6/50
897/897 ━━━━━━━━━━━━━━━━━━━━ 0s 41us/step - accuracy: 0.2812 - loss: 1.9134 - val_accuracy: 0.2000 - val_loss: 1.8097
Epoch 7/50
897/897 ━━━━━━━━━━━━━━━━━━━━ 49s 55ms/step - accuracy: 0.2838 - loss: 1.7387 - val_accuracy: 0.3525 - val_loss: 1.6305
Epoch 8/50
897/897 ━━━━━━━━━━━━━━━━━━━━ 0s 29us/step - accuracy: 0.3125 - loss: 1.8735 - val_

In [ ]:
# Load the best model
new_model = model.load_weights('best_model.keras')

In [ ]:
emotion_labels = list(train_generator.class_indices.keys())

In [68]:
import cv2
import numpy as np
import tensorflow as tf

# Load the pre-trained model with error handling
try:
    new_model = tf.keras.models.load_model('best_model.keras')
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")
    new_model = None

if new_model is None:
    raise ValueError("Model loading failed. Please check the model path and file.")

# Initialize webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Cannot open webcam")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break
    
    faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray, 1.1, 4)
    
    for (x, y, w, h) in faces:
        roi_color = frame[y:y+h, x:x+w]
        
        # Preprocess the face ROI for the model
        face_roi = cv2.resize(roi_color, (48, 48))  # Resize to match model input
        face_roi = np.expand_dims(face_roi, axis=0)  # Add batch dimension
        face_roi = face_roi / 255.0  # Normalize the image
        
        # Predict the emotion
        predictions = new_model.predict(face_roi)
        
        # Define the emotion labels
        emotion_labels = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]
        status = emotion_labels[np.argmax(predictions)]

        # Display emotion on the frame
        cv2.rectangle(frame, (x, y - 40), (x + w, y), (0, 0, 0), -1)
        cv2.putText(frame, status, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
    
    cv2.imshow('Face Emotion Recognition', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(2) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Model loaded successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━

KeyboardInterrupt: 